Online Webstore Dataset Analysis

In this notebook, you will find commands to read in 3 datasets that help an online webstore track customer sales behaviour.

The 3 main datasets are:

Customer: A dataset containing basic customer information <br>
Product: A dataset containing information about specific products and their prices <br>
Sales: A dataset containing information about purchases made by a customer <br>

We have put in examples for each of the questions to give you an idea of how you might complete it. These solutions are wrong and you will need to correct them or change them altogether in order to obtain the correct result.


In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType, ArrayType, StringType, DateType
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = (
    SparkSession.builder.appName("Spark Example").getOrCreate()
)

In [13]:

# Define the schema
sales_schema = StructType([
    StructField("SaleID", IntegerType(), True),
    StructField("CustomerID", IntegerType(), True),
    StructField("ProductIDs", ArrayType(IntegerType()), True),
    StructField("Purchase_Date", StringType(), True)  # Using StringType for simplicity
])


sales = spark.read.parquet('sales_dataset.parquet', schema =sales_schema,header=True)
sales.show()

+------+----------+--------------------+-------------+
|SaleID|CustomerID|          ProductIDs|Purchase_Date|
+------+----------+--------------------+-------------+
|   501|       297| [90, 72, 44, 4, 48]|   2023-02-20|
|   502|       493|        [57, 53, 52]|   2023-02-14|
|   503|       471|        [19, 62, 21]|   2023-11-10|
|   504|       127|                [42]|   2023-06-10|
|   505|       413|    [29, 97, 53, 80]|   2023-12-15|
|   506|       459|    [74, 64, 93, 15]|   2023-09-23|
|   507|       434|    [98, 33, 15, 28]|   2023-04-27|
|   508|       241|        [56, 57, 97]|   2023-02-06|
|   509|       366|[39, 61, 69, 95, 88]|   2023-11-03|
|   510|       261|                [77]|   2023-06-14|
|   511|       164|                [14]|   2023-11-29|
|   512|       350|   [94, 70, 52, 100]|   2023-11-25|
|   513|       369|                [63]|   2023-12-20|
|   514|       196|                [14]|   2023-10-13|
|   515|       021|  [5, 31, 8, 70, 98]|   2023-04-09|
|   516|  

<h5> Question 1: Based on how we have read in the schema for the sales dataset, can you apply a schema to the customer and product datasets? </h5>

In [14]:
customer = spark.read.parquet('customer_dataset.parquet', header=True)

In [15]:
customer.show()

+----------+-----------+----------+------------+--------------------+
|CustomerId|       Name|       DoB|Phone Number|               Email|
+----------+-----------+----------+------------+--------------------+
|       251|Customer251|1986-08-17| 01233756256|customer251@examp...|
|       252|Customer252|1972-03-15| 00226581961|customer252@examp...|
|       253|Customer253|1974-06-01| 55434224652|customer253@examp...|
|       254|Customer254|1995-07-16| 80703660494|customer254@examp...|
|       255|Customer255|1970-05-13| 92648819732|customer255@examp...|
|       256|Customer256|1981-02-05| 44807391583|customer256@examp...|
|       257|Customer257|1986-02-12| 50850238732|customer257@examp...|
|       258|Customer258|1972-11-17| 59516227714|customer258@examp...|
|       259|Customer259|1981-05-08| 12381763012|customer259@examp...|
|       260|Customer260|1979-08-27| 22794996604|customer260@examp...|
|       261|Customer261|1991-02-07| 79485574537|customer261@examp...|
|       262|Customer

In [16]:
product = spark.read.parquet('product_dataset.parquet', header=True)

In [17]:
product.show()

+---------+---------+-----+
|ProductID|     Item| cost|
+---------+---------+-----+
|       51|Product51|22.39|
|       52|Product52|30.01|
|       53|Product53|36.27|
|       54|Product54|16.06|
|       55|Product55|82.08|
|       56|Product56|53.73|
|       57|Product57|23.21|
|       58|Product58|72.54|
|       59|Product59|61.59|
|       60|Product60|24.33|
|       61|Product61|83.69|
|       62|Product62|57.67|
|       63|Product63|40.62|
|       64|Product64|15.57|
|       65|Product65|50.09|
|       66|Product66| 99.6|
|       67|Product67|28.77|
|       68|Product68|50.54|
|       69|Product69|85.06|
|       70|Product70| 64.1|
+---------+---------+-----+
only showing top 20 rows


In [18]:
# Register the DataFrame as a SQL temporary view
customer.createOrReplaceTempView("tmp_customer")
sales.createOrReplaceTempView("tmp_sales")
product.createOrReplaceTempView("tmp_product")

<h5> Question 2: Calculate the number of customers that have signed up to the website that have made a purchase on the website </h5>


In [19]:
spark.sql("""
          SELECT 
            count(c.customerid) 
          FROM tmp_customer AS c """).show()

+-----------------+
|count(customerid)|
+-----------------+
|              500|
+-----------------+



<h5> Question 3: Please generate the total sales figure for each product. <br>
As an example, product 1 might have 4 sales that it was part of, at £25 each, so we would want £100 to be output for that product  - Hint: Make use of the explode function </h5>

In [20]:

spark.sql("""
         SELECT 
            sum(cost) 
         FROM tmp_product AS p 
         JOIN tmp_sales AS s ON element_at(s.productIDs, 1) = p.productID 
         GROUP BY p.productID
          """).show()


+------------------+
|         sum(cost)|
+------------------+
|           1140.24|
|           1369.48|
|450.81000000000006|
|           1304.66|
|192.71999999999997|
|            873.27|
|             758.1|
|             126.0|
|             352.5|
|             968.4|
|            189.63|
|             92.84|
|            180.53|
|            552.32|
|285.01000000000005|
|311.43999999999994|
|453.39000000000016|
| 546.3899999999999|
|            517.02|
|290.43000000000006|
+------------------+
only showing top 20 rows


<h5> Question 4: Please Generate the date that a customer made their second purchase on the website. Hint: Make use of the Row_Number function </h5>

In [21]:
spark.sql("""
          SELECT 
            CustomerID, 
            FIRST_VALUE(Purchase_Date) OVER (PARTITION BY CustomerID ORDER BY Purchase_Date Desc) AS FirstPurchase 
          FROM tmp_sales
          """).show()


+----------+-------------+
|CustomerID|FirstPurchase|
+----------+-------------+
|       001|   2023-10-23|
|       001|   2023-10-23|
|       001|   2023-10-23|
|       001|   2023-10-23|
|       002|   2023-11-16|
|       002|   2023-11-16|
|       003|   2023-10-15|
|       003|   2023-10-15|
|       003|   2023-10-15|
|       003|   2023-10-15|
|       004|   2023-08-12|
|       004|   2023-08-12|
|       004|   2023-08-12|
|       005|   2023-12-30|
|       005|   2023-12-30|
|       005|   2023-12-30|
|       005|   2023-12-30|
|       007|   2023-08-11|
|       007|   2023-08-11|
|       007|   2023-08-11|
+----------+-------------+
only showing top 20 rows
